<a href="https://colab.research.google.com/github/yamaguchinorimitsu/ajax_app/blob/master/%E3%82%B9%E3%83%83%E3%82%AD%E3%83%AA%E6%A9%9F%E6%A2%B0%E7%AC%AC%EF%BC%99%E7%AB%A0%E7%AD%94%E3%81%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from sklearn import tree
from sklearn.model_selection import train_test_split
%matplotlib inline 

df = pd.read_csv('/content/Bank.csv')
print(df.shape)
df.head()

(23215, 16)


,id,age,job,marital,education,default,amount,housing,loan,contact,day,month,duration,campaign,previous,y
0,1,39,blue-collar,married,secondary,no,1756.0,yes,no,cellular,3.0,apr,370.055237,1.0,0.0,1.0
1,2,51,entrepreneur,married,primary,no,1443.0,no,no,cellular,18.0,feb,233.998933,10.0,0.0,1.0
2,3,36,management,single,tertiary,no,436.0,no,no,cellular,13.0,apr,NaN,1.0,2.0,0.0
3,4,63,retired,married,secondary,no,474.0,no,no,cellular,25.0,jan,252.525808,1.0,0.0,0.0
4,5,31,management,single,tertiary,no,354.0,no,no,cellular,30.0,apr,NaN,1.0,2.0,0.0


In [3]:
# まず、ダミー変数化をしたいが、文字列の列が複数あるので抜き出す。
str_col_name=['job','default','marital','education','housing','loan','contact','month']
str_df = df[str_col_name]
#複数列を一気にダミー変数化
str_df2=pd.get_dummies(str_df,drop_first=True)

num_df = df.drop(str_col_name,axis=1)#数値列を抜き出す
df2 = pd.concat([num_df,str_df2,str_df],axis=1)#結合(今後の集計の利便性も考慮してstr_dfも結合しておく)

In [4]:
df2.columns

Index(['id', 'age', 'amount', 'day', 'duration', 'campaign', 'previous', 'y',
       'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'default_yes', 'marital_married', 'marital_single',
       'education_secondary', 'education_tertiary', 'education_unknown',
       'housing_yes', 'loan_yes', 'contact_sending _document',
       'contact_telephone', 'month_aug', 'month_dec', 'month_feb', 'month_jan',
       'month_jul', 'month_jun', 'month_mar', 'month_may', 'month_nov',
       'month_oct', 'month_sep', 'job', 'default', 'marital', 'education',
       'housing', 'loan', 'contact', 'month'],
      dtype='object')

In [18]:
#訓練&検証データとテストデータに分割
train_val,test = train_test_split(df2,test_size=0.1,random_state=9)
train_val.head()

,id,age,amount,day,duration,campaign,previous,y,job_blue-collar,job_entrepreneur,...,month_oct,month_sep,job,default,marital,education,housing,loan,contact,month
5033,5034,46,174.0,8.0,343.311717,1.0,0.0,0.0,0,0,...,0,0,management,no,married,secondary,yes,no,cellular,may
4547,4548,32,329.0,17.0,260.329804,1.0,0.0,0.0,0,0,...,0,0,student,no,single,secondary,yes,no,cellular,apr
19987,19988,34,2918.0,4.0,267.490412,1.0,2.0,0.0,0,0,...,0,0,services,no,married,secondary,no,yes,cellular,jun
14680,14681,52,425.0,28.0,408.593120,2.0,3.0,1.0,0,0,...,0,0,technician,no,married,unknown,yes,no,cellular,jan
926,927,58,111.0,22.0,280.108939,1.0,2.0,0.0,0,0,...,0,0,management,no,married,secondary,no,no,telephone,jun


In [25]:
a=train_val.isnull().sum()
a[a>1]

duration    5434
dtype: int64

In [26]:
train_val['duration'].median()

315.64276813347817

In [27]:
#とりあえず最初は中央値で補完
train_val2= train_val.fillna(train_val.median())

<ipython-input-27-9e2d5178499f>:2: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  train_val2= train_val.fillna(train_val.median())


In [28]:
#不均衡なデータであるか確認
train_val2['y'].value_counts()

0.0    14248
1.0     6645
Name: y, dtype: int64

In [29]:
#とりあえず、全ての特徴量を利用してモデルを作ってみる。
t =train_val2['y']
x = train_val2.drop(str_col_name,axis=1)
x =x.drop(['id','y','day'],axis=1)

x_train,x_val,y_train,y_val = train_test_split(x,t,test_size=0.2,random_state=13)

#不均衡データに対応できるように、class_weight引数も設定
model = tree.DecisionTreeClassifier(random_state=3,max_depth=3,class_weight='balanced')

#class_weightを指定しないとちなみに正解率は0.7ぐらい
#model = tree.DecisionTreeClassifier(random_state=3,max_depth=5)
model.fit(x_train,y_train)
model.score(x_val,y_val)

0.661641541038526

In [35]:
# def learn(x,t,depth=3):
#     x_train,x_test,y_train,y_test = train_test_split(x,t,test_size = 0.2,random_state = 0)
#     model = tree.DecisionTreeClassifier(max_depth =depth,random_state = 0,class_weight="balanced")
#     model.fit(x_train,y_train)

#     #訓練データでも正解率を出している
#     score=model.score(X=x_train,y=y_train)

#     #テストデータで正解率を出している
#     score2=model.score(X=x_test,y=y_test)
#     return round(score,3),round(score2,3),model



# for j in range(1,20): # jは木の深さ jには1～14が入る
#     # xは特徴量、tは正解データ
#     train_score,test_score,model = learn(x,t,depth = j)
#     sentence="訓練データの正解率{}"
#     sentence2="テストデータの正解率{}"
#     total_sentence='深さ{}:'+sentence+sentence2
#     print(total_sentence.format(j,train_score,test_score))


In [36]:
#まず、さくっと学習できるようなlearn関数を定義する。
def learn(x,t,i):
    x_train,x_val,y_train,y_val = train_test_split(x,t,test_size=0.2,random_state=13)

    datas=[x_train,x_val,y_train,y_val]
    #不均衡データに対応できるように、class_weight引数も設定
    model = tree.DecisionTreeClassifier(random_state=i,max_depth=i,class_weight='balanced')
    model.fit(x_train,y_train)
    train_score=model.score(x_train,y_train)
    
    val_score=model.score(x_val,y_val)
    return train_score,val_score,model,datas

    #とりあえず、for文で様々な木の深さでの正解率を調べてみる
for i in range(1,20):
    s1,s2,model,datas = learn(x,t,i)
    print(i,s1,s2)

1 0.7123967931075745 0.7039961713328547
2 0.6616010530094532 0.661641541038526
3 0.6616010530094532 0.661641541038526
4 0.7307646284551873 0.7291217994735583
5 0.7441067368672969 0.7315147164393395
6 0.8132104822304654 0.7989949748743719
7 0.8270910613856647 0.806652309164872
8 0.8501256431733876 0.8262742282842785
9 0.8634079215029317 0.8167025604211534
10 0.8802201747038411 0.8181383106006221
11 0.8932033026205576 0.8224455611390284
12 0.9134857006102668 0.8183776022972002
13 0.9294005025726936 0.8092845178272314
14 0.9446571736268996 0.8092845178272314
15 0.9585975828646643 0.808566642737497
16 0.9708627497905947 0.810241684613544
17 0.9787603206892426 0.8078487676477626
18 0.984982649276056 0.8037808088059344
19 0.9909058274500419 0.8061737257717158


In [37]:
#深さ11以降はvalが停滞するので過学習が発生ここでテストデータでチェック
model = tree.DecisionTreeClassifier(max_depth=11,random_state=11)
model.fit(x,t)
test2 = test.copy()
test2=test2.fillna(train_val.median())

test_y=test2['y']
test_x = test2.drop(str_col_name,axis=1)
test_x =test_x.drop(['id','y','day'],axis=1)
model.score(test_x,test_y)

<ipython-input-37-16e81aa1e0c5>:5: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  test2=test2.fillna(train_val.median())


0.8458225667527993

In [38]:
#特徴量重要度の確認(ちなみに、連続量とダミー変数を比べると、連続量の方が重要度は高め出力されてしまう事に注意)
a=pd.Series(model.feature_importances_,index=x.columns).sort_values(ascending=False)
a[0:9]#campaignやdurationが重要度が大きいと分かる

duration                     0.530019
age                          0.139243
campaign                     0.100445
housing_yes                  0.059564
loan_yes                     0.041910
amount                       0.040618
contact_sending _document    0.015278
previous                     0.014210
month_jul                    0.004883
dtype: float64